In [12]:
#Enklar deklaratioer
m= 1000     #(kg) är massan
a_max= 29.4 #(m/s^2) Max acceleration innan slieringen

Cd= 0.3     #luftmotståndskoeffecient ENHETSLÖS!
Af= 1.63    #Area front ENHETSLÖS!
p_air=1.2   #(kg/m^2)   luftdensitet 
v_veh=0     #(m/s) Hastigheten som vi skall variera

C_rr= 0.3   #rullmotståndskoffecient 
alpha= 0    #(grader) lutningen i grander
g= 9.81     #(m/s^2) tyngdfaktor


In [15]:
#Implementering av ekvationerna som vi har från del 2
F_veh= m * a_max    
print(F_veh)

def F_aero(v_veh):
    F_aero= (Cd * Af * p_air * (v_veh^2)) / 2 
print(F_aero(v_veh))

def F_roll(alpha):
    F_roll= C_rr * m * g * cos(alpha)
    if alpha==0:
        return F_roll 
print(F_roll(alpha))




29400.0
None


NameError: name 'cos' is not defined